In [1]:
import pandas as pd

filtering = False

# Save to Excel
excel_path = 'RESULTS/episode_all_operator_action_plots/episodes_all_with_actions_and_deviations.xlsx'

episode_stats_df = pd.read_excel(excel_path, engine='openpyxl')
episode_stats_df.head()

,EpisodeID,AlarmStart,AlarmEnd,AlarmDurationMinutes,TotalWindowMinutes,OperatedTags,OperatedTagsCount,DeviatedTags,DeviatedTagsCount,HasOperatorActions,HasOnlyTargetTags,Has1071Action,Has1016Action,Has1013Action
0,1,2022-01-05 08:53:00,2022-01-05 09:33:00,40,186,"03FIC_1085, 03FIC_3435, 03HIC_1141, 03HIC_1151...",15,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",22,True,False,True,True,True
1,2,2022-01-07 09:55:00,2022-01-07 10:00:00,5,151,"03FIC_3435, 03GHS_0121A, 03GHS_0121AA, 03GHS_0...",8,"03FIC_1085.PV, 03FIC_3415.PV, 03FI_1141A.PV, 0...",23,True,False,False,False,False
2,3,2022-01-07 13:33:00,2022-01-07 13:36:00,3,149,"03FIC_3435, 03LIC_1034",2,"03FIC_1085.PV, 03FIC_3415.PV, 03FI_1141A.PV, 0...",19,True,False,False,False,False
3,4,2022-01-07 14:17:00,2022-01-07 14:19:00,2,148,"03FIC_3435, 03LIC_1016, 03LIC_1034",3,"03FIC_1085.PV, 03FI_1141A.PV, 03LIC_1016.PV, 0...",22,True,False,False,True,False
4,5,2022-01-07 14:54:00,2022-01-07 14:58:00,4,150,"03FIC_3435, 03LIC_1016",2,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1141A.PV, 03...",19,True,False,False,True,False


In [2]:
# Run this code to filter operated tags based on KG filter otherwise comment it out

filtering = True
kg_filter_excel_path = "/home/h604827/ControlActions/DATA/03LIC_1071_Tag_result_03Feb.xlsx"
kg_filter_df = pd.read_excel(kg_filter_excel_path, engine='openpyxl', sheet_name='03LIC_1071-operatortag')

# Build allowed tag set from TagName column
allowed_tags = {str(tag).strip() for tag in kg_filter_df['TagName'].dropna()}

def filter_operated_tags(tags_str):
    if pd.isna(tags_str):
        return tags_str
    tags = [tag.strip() for tag in str(tags_str).split(',') if tag.strip()]
    filtered = [tag for tag in tags if tag in allowed_tags]
    return ', '.join(filtered) if filtered else None

# Calculate statistics before filtering
original_count = episode_stats_df['OperatedTags'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) else 0).sum()

# Compute all_tags from original OperatedTags before filtering
operated_series_original = episode_stats_df['OperatedTags'].dropna()

def parse_operated_tags(tags_str):
    return {tag.strip() for tag in str(tags_str).split(',') if tag.strip()}

episode_tag_sets_original = operated_series_original.apply(parse_operated_tags)
all_tags = sorted({tag for tag_set in episode_tag_sets_original for tag in tag_set})

# Keep only tags that appear in TagName (overwrite original column)
episode_stats_df['OperatedTags'] = episode_stats_df['OperatedTags'].apply(filter_operated_tags)

# Calculate statistics after filtering
filtered_count = episode_stats_df['OperatedTags'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) and x != '' else 0).sum()
removed_count = original_count - filtered_count

print(f"Original operated tags count: {original_count}")
print(f"Filtered operated tags count: {filtered_count}")
print(f"Tags removed: {removed_count} ({removed_count/original_count*100:.1f}%)")
print(f"\nAllowed tags from KG filter: {len(allowed_tags)}")
print(f"Unique tags in original OperatedTags: {len(all_tags)}")

removed_tags = set(all_tags) - allowed_tags
kg_specific_tags = allowed_tags - set(all_tags)
common_tags = allowed_tags & set(all_tags)

print(f"\nRemoved tags: {removed_tags}")
print(f"Number of removed tags: {len(removed_tags)}")
print(f"Number of common tags: {len(common_tags)}")
print(f"Common tags: {sorted(common_tags)}")
print(f"\nKG specific tags not in OperatedTags: {kg_specific_tags}")
print(f"Number of KG specific tags (in KG but not in OperatedTags): {len(kg_specific_tags)}")

episode_stats_df[['OperatedTags']].head(10)

Original operated tags count: 2219
Filtered operated tags count: 1354
Tags removed: 865 (39.0%)

Allowed tags from KG filter: 111
Unique tags in original OperatedTags: 115

Removed tags: {'03SDV_3159', '03XAX_1020', '03HIC_1141', '03XAX_1002', '03LI_1035_MOS', '03GHS_1511BA', '03GHS_0153BA', '03PDI_0011_MOS', '03HIC_1183', '03GM_0114A', '03XAX_1479', '03GM_0112', '03GHS_0151AA', '03SDV_3105', '03HIC_3100', '03GM_1511A', '03SDV_3101', '03LI_1128_MOS', '03GHS_0121B', '03GM_0152B', '03GHS_0112A', '03GM_1511B', '03GM_0121A', '03XAX_1428OR', '03GM_0114', '03GHS_0112AA', '03GM_0152A', '03XAX_1220_SEL', '03PDI_0018_MOS', '03PDI_0016_MOS', '03HIC_1023B', '03LI_1086_MOS', '03TI_0045A_MOS', '03VS_0002', '03GM_0121', '03SDV_1201', '03HIC_1023A', '03GHS_0121AA', '03PAX_1012', '03KT_0151', '03GM_0153B', '03LI_1174_MOS', '03TI_0042A_MOS', '03HIC_1151', '03PDI_0007_MOS', '03LI_3151_MOS', '03EHS_1511BA', '04RES_HYP1COM', '03XAX_1027', '03TI_0019_MOS', '03GM_0153C', '03XAX_1013', '03XAX_1001', '03GHS_0

,OperatedTags
0,"03FIC_1085, 03FIC_3435, 03LIC_1016, 03LIC_1031..."
1,03FIC_3435
2,"03FIC_3435, 03LIC_1034"
3,"03FIC_3435, 03LIC_1016, 03LIC_1034"
4,"03FIC_3435, 03LIC_1016"
5,03FIC_3435
6,03FIC_3435
7,03FIC_3435
8,03FIC_3435
9,03FIC_3435


In [3]:
# Episode counts per operated tag (unique per episode)
operated_series = episode_stats_df['OperatedTags'].dropna()

def parse_operated_tags(tags_str):
    return {tag.strip() for tag in tags_str.split(',') if tag.strip()}

# Build a per-episode set of tags, then count episodes per tag
episode_tag_sets = operated_series.apply(parse_operated_tags)
all_tags = sorted({tag for tag_set in episode_tag_sets for tag in tag_set})

tag_episode_counts = {
    tag: int(episode_tag_sets.apply(lambda s: tag in s).sum())
    for tag in all_tags
}

tag_episode_counts_df = (
    pd.DataFrame.from_dict(tag_episode_counts, orient='index', columns=['EpisodeCount'])
      .sort_values('EpisodeCount', ascending=False)
      .reset_index()
      .rename(columns={'index': 'Tag'})
)

tag_episode_counts_df

,Tag,EpisodeCount
0,03FIC_3435,353
1,03PIC_1013,173
2,03LIC_1034,163
3,03LIC_1071,104
4,03LIC_1085,76
5,03TIC_1009,60
6,03FIC_3415,45
7,03LIC_1016,45
8,03LIC_3153,34
9,03PIC_3131,34


In [4]:
path = 'RESULTS/episode_all_operator_action_plots/tag_episode_counts_filtered.csv' if filtering else 'RESULTS/episode_all_operator_action_plots/tag_episode_counts.csv'

tag_episode_counts_df.to_csv(path, index=False)

In [5]:
target_tags_to_filter = ['03LIC_1016', '03LIC_1071', '03PIC_1013', '03FIC_3435']

episode_stats_df[
    episode_stats_df['OperatedTags'].notna() &
    episode_stats_df['OperatedTags'].str.contains('|'.join(target_tags_to_filter), regex=True)
].shape

(450, 14)

In [6]:
episode_stats_df[episode_stats_df['OperatedTags'].notna() & episode_stats_df['OperatedTags'].str.contains('03LIC_1016')].shape

(45, 14)

In [7]:
episode_stats_df[episode_stats_df['OperatedTags'].notna() & episode_stats_df['OperatedTags'].str.contains('03LIC_1071')].shape

(104, 14)

In [8]:
episode_stats_df[episode_stats_df['OperatedTags'].notna() & episode_stats_df['OperatedTags'].str.contains('03PIC_1013')].shape

(173, 14)

In [9]:
target_tags = {'03LIC_1016', '03LIC_1071', '03PIC_1013', '03FIC_3435'}

def contains_only_target_tags(operated_tags_str):
    if pd.isna(operated_tags_str):
        return False
    # Split by comma and strip whitespace
    tags = {tag.strip() for tag in operated_tags_str.split(',')}
    # Check if tags is a subset of target_tags (and not empty)
    return len(tags) > 0 and tags.issubset(target_tags)

episode_stats_df[
    episode_stats_df['OperatedTags'].apply(contains_only_target_tags)
]

,EpisodeID,AlarmStart,AlarmEnd,AlarmDurationMinutes,TotalWindowMinutes,OperatedTags,OperatedTagsCount,DeviatedTags,DeviatedTagsCount,HasOperatorActions,HasOnlyTargetTags,Has1071Action,Has1016Action,Has1013Action
1,2,2022-01-07 09:55:00,2022-01-07 10:00:00,5,151,03FIC_3435,8,"03FIC_1085.PV, 03FIC_3415.PV, 03FI_1141A.PV, 0...",23,True,False,False,False,False
4,5,2022-01-07 14:54:00,2022-01-07 14:58:00,4,150,"03FIC_3435, 03LIC_1016",2,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1141A.PV, 03...",19,True,False,False,True,False
5,6,2022-01-07 19:56:00,2022-01-07 20:02:00,6,152,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",21,True,False,False,False,False
6,7,2022-01-07 23:28:00,2022-01-07 23:31:00,3,149,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1141A.PV, 03...",22,True,False,False,False,False
7,8,2022-01-10 10:09:00,2022-01-10 10:15:00,6,152,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",19,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,603,2025-06-21 15:47:00,2025-06-21 20:28:00,281,427,"03LIC_1071, 03PIC_1013",4,"03FIC_1085.PV, 03FIC_3415.PV, 03FI_1151.PV, 03...",20,True,False,True,False,True
603,604,2025-06-21 22:15:00,2025-06-21 22:16:00,1,147,03LIC_1071,1,"03FIC_1085.PV, 03FIC_3415.PV, 03LIC_1016.PV, 0...",12,True,True,True,False,False
604,605,2025-06-22 13:13:00,2025-06-22 13:19:00,6,152,"03FIC_3435, 03PIC_1013",3,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",22,True,False,False,False,True
605,606,2025-06-22 13:36:00,2025-06-22 13:50:00,14,160,"03FIC_3435, 03PIC_1013",3,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1151.PV, 03L...",19,True,False,False,False,True


In [10]:
episode_stats_df[episode_stats_df['HasOnlyTargetTags'] == True].shape

(27, 14)

In [11]:
events_df = pd.read_csv('/home/h604827/ControlActions/DATA/trip_filtered_events.csv', low_memory=False)
actions_df = events_df[(events_df['ConditionName'] == 'CHANGE') & (events_df['Description'] == 'OP')]
actions_df.shape

(162020, 29)

In [12]:
actions_df_filtered = actions_df[actions_df['Source'].isin(common_tags)]
actions_df_filtered.shape

(91298, 29)

In [13]:
op_pv_data = pd.read_parquet('/home/h604827/ControlActions/DATA/03LIC_1071_JAN_2026_filtered.parquet')
op_pv_data.head()

,03LIC_1071.PV,03LIC_1071.OP,02FI_1000.PV,03FIC_1085.OP,03FIC_1085.PV,03FIC_3415.OP,03FIC_3415.PV,03FIC_3435.PV,03FI_1141A.PV,03FI_1151.PV,...,03TIC_1142.OP,03TIC_1142.PV,03TIC_1145.OP,03TIC_1145.PV,03TI_1015.PV,03TI_1081.PV,03TI_1421.PV,03TI_1901.PV,AlarmStatus,AlarmType
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2022-01-03 22:45:00,36.399925,41.573322,6.645525,29.419975,216.82233,NaN,43.197433,24373.373,64872.184,227702.23,...,NaN,-39.511604,NaN,34.514153,16.732132,-36.141970,-8.287849,-34.645950,OFF,
2022-01-03 22:46:00,37.009620,41.312057,6.682778,29.780640,210.55800,NaN,44.147410,19615.037,65406.332,222237.81,...,NaN,-39.511604,NaN,34.443573,16.748703,-36.198616,-8.386974,-34.684036,OFF,
2022-01-03 22:47:00,37.161285,41.206524,6.713386,29.082687,212.94440,NaN,43.628166,19882.620,64685.234,225012.92,...,NaN,-39.539840,NaN,34.415344,16.665878,-36.283580,-8.386974,-34.722122,OFF,
2022-01-03 22:48:00,37.534748,40.972713,6.706837,29.054146,213.07224,NaN,43.698970,19952.428,65085.844,222123.38,...,NaN,-39.497490,NaN,34.274178,16.583061,-36.297745,-8.273697,-34.817337,OFF,
2022-01-03 22:49:00,37.370754,40.977924,6.670256,28.769829,214.64897,NaN,43.639970,19917.525,64044.258,222581.12,...,NaN,-39.539840,NaN,34.373000,16.616188,-36.241100,-8.103760,-34.817337,OFF,


# Correlation Analysis: Impact of Operator Actions on 03LIC_1071

We analyze the correlation between 03LIC_1071.PV and other tags (PV and OP) in a future window after operator actions.

**Approach:**
- Use Spearman correlation (rank-based) since time series data is non-linear
- Consider multiple future windows (10, 15, 20 minutes)
- Analyze per year for consistency check

In [14]:
from scipy.stats import spearmanr
import numpy as np

# Prepare data
actions_df['VT_Start'] = pd.to_datetime(actions_df['VT_Start'])
actions_df['VT_Start_rounded'] = actions_df['VT_Start'].dt.round('min')

# Ensure op_pv_data has datetime index
if not isinstance(op_pv_data.index, pd.DatetimeIndex):
    op_pv_data['TimeStamp'] = pd.to_datetime(op_pv_data['TimeStamp'])
    op_pv_data.set_index('TimeStamp', inplace=True)
op_pv_data.sort_index(inplace=True)

# Add year column to actions
actions_df['Year'] = actions_df['VT_Start'].dt.year

print(f"Actions data range: {actions_df['VT_Start'].min()} to {actions_df['VT_Start'].max()}")
print(f"OP/PV data range: {op_pv_data.index.min()} to {op_pv_data.index.max()}")
print(f"\nActions per year:")
print(actions_df['Year'].value_counts().sort_index())

Actions data range: 2021-10-02 08:49:05.611200 to 2025-06-27 22:33:19.875700
OP/PV data range: 2022-01-03 22:45:00 to 2025-06-23 20:44:00

Actions per year:
Year
2021    13348
2022    78792
2023    35152
2024    20924
2025    13804
Name: count, dtype: int64


/tmp/ipykernel_798549/1170987143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_df['VT_Start'] = pd.to_datetime(actions_df['VT_Start'])
/tmp/ipykernel_798549/1170987143.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_df['VT_Start_rounded'] = actions_df['VT_Start'].dt.round('min')
/tmp/ipykernel_798549/1170987143.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [15]:
# Load SSD file to get alarm episode windows
ssd_path = '/home/h604827/ControlActions/DATA/SSD_1071_SSD_output_1071_7Jan2026.xlsx'
ssd_df = pd.read_excel(ssd_path, engine='openpyxl')

print(f"SSD data shape: {ssd_df.shape}")
print(f"Columns: {ssd_df.columns.tolist()}")
ssd_df.head()

SSD data shape: (10107, 31)
Columns: ['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 'TagName', 'Units', 'Tag_First_Transition_Start_minutes', 'Latest_Transition_Start_Time_minutes', 'Latest_Transition_End_Time_minutes', 'PV_at_latest_transition_start', 'PV_at_latest_transition_end', 'Max_Change_within_transition', 'Slope_degree', 'Slope_Post_Transition_degree', 'Rate_of_Change_entire_transition_window', 'Volatility', 'Slope_degree_Standardized', 'Slope_Post_Transition_degree_Standardized', 'Rate_of_Change_entire_transition_window_Standardized', 'Transition_Change_Percent', 'MaxChangePercent_from_LatestTransitionStart_to_Alarm', 'MaxChangeStartTime', 'MaxChangeEndTime', 'MaxChangeStartValue', 'MaxChangeEndValue', 'MaxMinChangePercentageTransitionToAlarm', 'MaxValueTime', 'MaxValue', 'MinValueTime', 'MinValue', 'equipmentName', 'Depth', 'Stream']


,AlarmStart_rounded_minutes,AlarmEnd_rounded_minutes,TagName,Units,Tag_First_Transition_Start_minutes,Latest_Transition_Start_Time_minutes,Latest_Transition_End_Time_minutes,PV_at_latest_transition_start,PV_at_latest_transition_end,Max_Change_within_transition,...,MaxChangeStartValue,MaxChangeEndValue,MaxMinChangePercentageTransitionToAlarm,MaxValueTime,MaxValue,MinValueTime,MinValue,equipmentName,Depth,Stream
0,2022-01-05 08:53:00,2022-01-05 09:33:00,03LIC_1071.PV,%,2022-01-05 08:36:00,2022-01-05 08:42:00,2022-01-05 08:52:00,55.615295,31.869987,8.069202,...,65.262160,29.133090,124.013862,2022-01-05 08:44:00,65.262160,2022-01-05 08:53:00,29.133090,03-E-107,2.0,UPSTREAM
1,2022-01-05 08:53:00,2022-01-05 09:33:00,02FI_1000.PV,T/Hr,2022-01-05 07:28:00,2022-01-05 08:44:00,2022-01-05 08:52:00,5.804195,6.008471,0.074640,...,5.804195,6.029510,3.881951,2022-01-05 08:50:00,6.029510,2022-01-05 08:44:00,5.804195,03-E-102,4.0,UPSTREAM
2,2022-01-05 08:53:00,2022-01-05 09:33:00,03FIC_1085.PV,SCMH,2022-01-05 08:37:00,2022-01-05 08:43:00,2022-01-05 08:52:00,242.092680,214.648970,86.251630,...,226.325330,313.173550,79.112818,2022-01-05 08:48:00,313.173550,2022-01-05 08:53:00,174.847090,03-E-106,3.0,UPSTREAM
3,2022-01-05 08:53:00,2022-01-05 09:33:00,03FIC_3415.PV,SCMH,2022-01-05 08:36:00,2022-01-05 08:41:00,2022-01-05 08:52:00,51.664623,49.269028,5.204227,...,56.685936,46.507603,21.885310,2022-01-05 08:45:00,56.685936,2022-01-05 08:50:00,46.507603,03-C-155,5.0,DOWNSTREAM
4,2022-01-05 08:53:00,2022-01-05 09:33:00,03FI_1151.PV,SCMH,2022-01-05 08:33:00,2022-01-05 08:37:00,2022-01-05 08:47:00,276882.000000,311842.800000,21914.880000,...,276882.000000,311842.800000,12.626606,2022-01-05 08:40:00,311842.800000,2022-01-05 08:37:00,276882.000000,03-K-101,5.0,SAME


In [16]:
# Build alarm episode windows
# Each unique (AlarmStart, AlarmEnd) pair is an episode
# Start = earliest Tag_First_Transition_Start_minutes, End = AlarmEnd_rounded_minutes + 60 min

# Group by alarm start/end pair and get earliest transition start
episode_windows = ssd_df.groupby(['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes']).agg({
    'Tag_First_Transition_Start_minutes': 'min'  # Earliest transition start across all tags
}).reset_index()

episode_windows.columns = ['AlarmStart', 'AlarmEnd', 'EpisodeStart']

# Convert to datetime if not already
episode_windows['AlarmStart'] = pd.to_datetime(episode_windows['AlarmStart'])
episode_windows['AlarmEnd'] = pd.to_datetime(episode_windows['AlarmEnd'])
episode_windows['EpisodeStart'] = pd.to_datetime(episode_windows['EpisodeStart'])

# Episode end = AlarmEnd + 60 minutes
episode_windows['EpisodeEnd'] = episode_windows['AlarmEnd'] + pd.Timedelta(minutes=60)

print(f"Total alarm episodes: {len(episode_windows)}")
print(f"\nEpisode windows sample:")
episode_windows.head(10)

Total alarm episodes: 609

Episode windows sample:


,AlarmStart,AlarmEnd,EpisodeStart,EpisodeEnd
0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
1,2022-01-07 09:55:00,2022-01-07 10:00:00,2022-01-07 08:29:00,2022-01-07 11:00:00
2,2022-01-07 13:33:00,2022-01-07 13:36:00,2022-01-07 12:07:00,2022-01-07 14:36:00
3,2022-01-07 14:17:00,2022-01-07 14:19:00,2022-01-07 12:51:00,2022-01-07 15:19:00
4,2022-01-07 14:54:00,2022-01-07 14:58:00,2022-01-07 13:28:00,2022-01-07 15:58:00
5,2022-01-07 19:56:00,2022-01-07 20:02:00,2022-01-07 18:30:00,2022-01-07 21:02:00
6,2022-01-07 23:28:00,2022-01-07 23:31:00,2022-01-07 22:02:00,2022-01-08 00:31:00
7,2022-01-10 10:09:00,2022-01-10 10:15:00,2022-01-10 08:43:00,2022-01-10 11:15:00
8,2022-01-12 10:41:00,2022-01-12 10:45:00,2022-01-12 09:15:00,2022-01-12 11:45:00
9,2022-01-14 11:28:00,2022-01-14 11:35:00,2022-01-14 10:02:00,2022-01-14 12:35:00


In [17]:
# Filter actions to only those during alarm episodes
# An action is during an episode if: EpisodeStart <= VT_Start <= EpisodeEnd

def is_action_during_episode(action_time, episode_windows):
    """Check if action time falls within any episode window."""
    if pd.isna(action_time):
        return False
    for _, ep in episode_windows.iterrows():
        if ep['EpisodeStart'] <= action_time <= ep['EpisodeEnd']:
            return True
    return False

# Vectorized approach using IntervalIndex for efficiency
from pandas import IntervalIndex

# Create interval index from episode windows
intervals = IntervalIndex.from_arrays(
    episode_windows['EpisodeStart'],
    episode_windows['EpisodeEnd'],
    closed='both'
)

# Check which actions fall within any episode
actions_df['InAlarmEpisode'] = actions_df['VT_Start'].apply(
    lambda t: any(intervals.contains(t)) if pd.notna(t) else False
)

# Filter to only actions during alarm episodes
actions_during_episodes = actions_df[actions_df['InAlarmEpisode']].copy()

print(f"Total actions: {len(actions_df)}")
print(f"Actions during alarm episodes: {len(actions_during_episodes)}")
print(f"Percentage: {len(actions_during_episodes)/len(actions_df)*100:.1f}%")
print(f"\nActions during episodes per year:")
print(actions_during_episodes['Year'].value_counts().sort_index())

Total actions: 162020
Actions during alarm episodes: 27914
Percentage: 17.2%

Actions during episodes per year:
Year
2022    8780
2023    7168
2024    7866
2025    4100
Name: count, dtype: int64


/tmp/ipykernel_798549/3028033830.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_df['InAlarmEpisode'] = actions_df['VT_Start'].apply(


In [18]:
actions_during_episodes[actions_during_episodes['Source'] == '03SDV_1167']

,Action,Actor,AreaName,AlarmLimit,Block,Category,ConditionName,Description,EventID,Flags,...,TransactionID,Units,Value,VT_Start,H,TagID,AlarmStatus,VT_Start_rounded,Year,InAlarmEpisode
147457,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,35635267,NaN,...,35635267,NaN,CLOSE,2022-01-05 08:48:46.908100,2022_01_05_10,NaN,NaN,2022-01-05 08:49:00,2022,True
147458,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,35635267,NaN,...,35635267,NaN,CLOSE,2022-01-05 08:48:46.908100,2022_01_05_10,03SDV_1167,NaN,2022-01-05 08:49:00,2022,True
148130,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,35643471,NaN,...,35643471,NaN,CLOSE,2022-01-05 09:30:00.003300,2022_01_05_11,NaN,NaN,2022-01-05 09:30:00,2022,True
148132,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,35643471,NaN,...,35643471,NaN,CLOSE,2022-01-05 09:30:00.003300,2022_01_05_11,03SDV_1167,NaN,2022-01-05 09:30:00,2022,True
190732,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,37318118,NaN,...,37318118,NaN,CLOSE,2022-02-07 13:38:38.253100,2022_02_07_15,NaN,NaN,2022-02-07 13:39:00,2022,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579673,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,78595641,NaN,...,78595641,NaN,CLOSE,2025-05-08 00:56:31.067500,2025_05_08_02,NaN,NaN,2025-05-08 00:57:00,2025,True
1581147,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,78617952,NaN,...,78617952,NaN,CLOSE,2025-05-09 11:02:37.307200,2025_05_09_12,NaN,NaN,2025-05-09 11:03:00,2025,True
1581148,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,78617952,NaN,...,78617952,NaN,CLOSE,2025-05-09 11:02:37.307200,2025_05_09_12,03SDV_1167,NaN,2025-05-09 11:03:00,2025,True
1581338,NaN,NaN,1F,NaN,NaN,7,CHANGE,OP,78618645,NaN,...,78618645,NaN,CLOSE,2025-05-09 11:29:09.071900,2025_05_09_12,NaN,NaN,2025-05-09 11:29:00,2025,True


In [19]:
actions_during_episodes.columns

Index(['Action', 'Actor', 'AreaName', 'AlarmLimit', 'Block', 'Category',
       'ConditionName', 'Description', 'EventID', 'Flags', 'LocalTime',
       'LocationFullName', 'LocationTagName', 'PrevValue', 'Priority',
       'ReceivedDelay', 'ServerName', 'ShelvedReason', 'Source',
       'SourceParameter', 'Station', 'Time', 'TransactionID', 'Units', 'Value',
       'VT_Start', 'H', 'TagID', 'AlarmStatus', 'VT_Start_rounded', 'Year',
       'InAlarmEpisode'],
      dtype='object')

In [20]:
# Focus on the 35 common tags (intersection of KG and OperatedTags)
# common_tags is already defined from cell 2

print(f"Number of common tags (KG ∩ OperatedTags): {len(common_tags)}")
print(f"Common tags: {sorted(common_tags)}")

# Check which common tags have PV and OP data in op_pv_data
op_pv_columns = set(op_pv_data.columns)

# Extract base tag names from op_pv_data columns
op_pv_base_tags = set()
for col in op_pv_columns:
    if col.endswith('.PV') or col.endswith('.OP'):
        op_pv_base_tags.add(col.rsplit('.', 1)[0])

print(f"\nUnique base tags in op_pv_data: {len(op_pv_base_tags)}")

# Check overlap with common_tags
common_in_op_pv = common_tags & op_pv_base_tags
common_not_in_op_pv = common_tags - op_pv_base_tags
op_pv_not_in_common = op_pv_base_tags - common_tags

print(f"\n{'='*60}")
print(f"Common tags WITH PV/OP data: {len(common_in_op_pv)}")
print(f"Common tags WITHOUT PV/OP data: {len(common_not_in_op_pv)}")
print(f"OP/PV tags NOT in common tags: {len(op_pv_not_in_common)}")

print(f"\n--- Common tags WITH PV/OP data ({len(common_in_op_pv)}) ---")
print(sorted(common_in_op_pv))

print(f"\n--- Common tags WITHOUT PV/OP data ({len(common_not_in_op_pv)}) ---")
print(sorted(common_not_in_op_pv))

print(f"\n--- OP/PV tags NOT in common tags ({len(op_pv_not_in_common)}) ---")
print(sorted(op_pv_not_in_common))

# For each common tag with data, check if it has PV, OP, or both
print(f"\n{'='*60}")
print("PV/OP availability for common tags:")
print("="*60)

tag_availability = []
for tag in sorted(common_in_op_pv):
    has_pv = f"{tag}.PV" in op_pv_columns
    has_op = f"{tag}.OP" in op_pv_columns
    tag_availability.append({
        'Tag': tag,
        'Has_PV': has_pv,
        'Has_OP': has_op,
        'Type': 'Both' if (has_pv and has_op) else ('PV only' if has_pv else 'OP only')
    })

tag_availability_df = pd.DataFrame(tag_availability)
print(tag_availability_df.to_string(index=False))

print(f"\nSummary:")
print(f"  Tags with both PV and OP: {(tag_availability_df['Type'] == 'Both').sum()}")
print(f"  Tags with PV only: {(tag_availability_df['Type'] == 'PV only').sum()}")
print(f"  Tags with OP only: {(tag_availability_df['Type'] == 'OP only').sum()}")

Number of common tags (KG ∩ OperatedTags): 35
Common tags: ['03FIC_1085', '03FIC_3172', '03FIC_3415', '03FIC_3435', '03FIC_3435A', '03HIC_1009A', '03HIC_1009B', '03HIC_1092A', '03HIC_1183A', '03LIC_1016', '03LIC_1031', '03LIC_1034', '03LIC_1071', '03LIC_1085', '03LIC_1094', '03LIC_1097', '03LIC_3153', '03LIC_3178', '03LIC_3408', '03PIC_1013', '03PIC_1023', '03PIC_1068', '03PIC_3131', '03SDV_1167', '03SDV_1202', '03SDV_1455', '03SDV_1456', '03SDV_3120', '03SDV_3196', '03SDV_3197', '03SDV_3198', '03SDV_3422', '03SDV_3450', '03TIC_1009', '03TIC_1023']

Unique base tags in op_pv_data: 28

Common tags WITH PV/OP data: 12
Common tags WITHOUT PV/OP data: 23
OP/PV tags NOT in common tags: 16

--- Common tags WITH PV/OP data (12) ---
['03FIC_1085', '03FIC_3415', '03FIC_3435', '03LIC_1016', '03LIC_1071', '03LIC_1085', '03LIC_1094', '03LIC_1097', '03LIC_3178', '03PIC_1013', '03PIC_1068', '03PIC_3131']

--- Common tags WITHOUT PV/OP data (23) ---
['03FIC_3172', '03FIC_3435A', '03HIC_1009A', '03HIC

In [21]:
# Build the list of columns to analyze - ONLY from common tags with data
target_pv = '03LIC_1071.PV'
target_op = '03LIC_1071.OP'

# Get PV and OP columns only for common tags that have data
focus_pv_cols = [f"{tag}.PV" for tag in common_in_op_pv if f"{tag}.PV" in op_pv_columns and f"{tag}.PV" != target_pv]
focus_op_cols = [f"{tag}.OP" for tag in common_in_op_pv if f"{tag}.OP" in op_pv_columns]

print(f"Target PV: {target_pv}")
print(f"Target OP: {target_op}")
print(f"\nFocus PV columns (from common tags): {len(focus_pv_cols)}")
print(focus_pv_cols)
print(f"\nFocus OP columns (from common tags): {len(focus_op_cols)}")
print(focus_op_cols)

# Columns to use in correlation analysis
cols_to_analyze = focus_op_cols + focus_pv_cols
print(f"\nTotal columns for correlation analysis: {len(cols_to_analyze)}")

Target PV: 03LIC_1071.PV
Target OP: 03LIC_1071.OP

Focus PV columns (from common tags): 11
['03PIC_3131.PV', '03LIC_1097.PV', '03LIC_1085.PV', '03PIC_1013.PV', '03FIC_1085.PV', '03FIC_3435.PV', '03PIC_1068.PV', '03LIC_1094.PV', '03LIC_1016.PV', '03FIC_3415.PV', '03LIC_3178.PV']

Focus OP columns (from common tags): 11
['03PIC_3131.OP', '03LIC_1097.OP', '03LIC_1085.OP', '03PIC_1013.OP', '03FIC_1085.OP', '03PIC_1068.OP', '03LIC_1094.OP', '03LIC_1016.OP', '03FIC_3415.OP', '03LIC_3178.OP', '03LIC_1071.OP']

Total columns for correlation analysis: 22


In [24]:
from scipy.stats import spearmanr
import numpy as np
from tqdm import tqdm

def build_episode_interval_index(episode_windows):
    """Build IntervalIndex from episode start/end columns."""
    return pd.IntervalIndex.from_arrays(
        episode_windows['EpisodeStart'],
        episode_windows['EpisodeEnd'],
        closed='both',
    )

def filter_op_pv_to_episode_windows(op_pv_data, episode_windows):
    """
    Vectorized filter of op_pv_data to rows that fall within any episode window.
    Returns filtered DataFrame and boolean mask.
    Handles overlapping episode windows.
    """
    intervals = build_episode_interval_index(episode_windows)
    # Use get_indexer_non_unique for overlapping intervals
    indexer, missing = intervals.get_indexer_non_unique(op_pv_data.index)
    # Create mask: True if timestamp is in any interval (not in missing)
    in_window = np.ones(len(op_pv_data), dtype=bool)
    in_window[missing] = False
    return op_pv_data.loc[in_window].copy(), in_window

def compute_episode_window_correlation(op_pv_subset, target_col, other_cols):
    """Compute Spearman correlation within episode windows for target vs other columns."""
    if target_col not in op_pv_subset.columns:
        return {}
    target_vals = op_pv_subset[target_col].dropna()
    if len(target_vals) < 10:
        return {}
    results = {}
    for col in tqdm(other_cols, desc="      Computing correlations", unit="col", leave=False):
        if col not in op_pv_subset.columns:
            continue
        col_vals = op_pv_subset[col].dropna()
        # Align by index
        common_idx = target_vals.index.intersection(col_vals.index)
        if len(common_idx) >= 10:
            corr, pval = spearmanr(target_vals.loc[common_idx], col_vals.loc[common_idx])
            results[col] = (corr, pval, len(common_idx))
    return results

print("Episode-window correlation helpers defined (IntervalIndex-based filtering with overlap support).")

Episode-window correlation helpers defined (IntervalIndex-based filtering with overlap support).


In [25]:
# Run correlation analysis per year using full alarm episode windows

# Filter OP/PV data to episode windows (vectorized)
episode_op_pv_data, episode_mask = filter_op_pv_to_episode_windows(op_pv_data, episode_windows)
print(f"Total OP/PV rows: {len(op_pv_data)}")
print(f"Rows within episode windows: {len(episode_op_pv_data)}")
print(f"Percent within episodes: {len(episode_op_pv_data)/len(op_pv_data)*100:.2f}%")

# Years with data inside episode windows
valid_years = sorted(episode_op_pv_data.index.year.unique())
print(f"Years with OP/PV data inside episodes: {valid_years}")

results_all = []

for year in valid_years:
    print(f"\n{'='*50}")
    print(f"Processing Year: {year}")
    print(f"{'='*50}")
    
    # Filter episode data for this year
    year_data = episode_op_pv_data[episode_op_pv_data.index.year == year].copy()
    print(f"Rows in episode windows for {year}: {len(year_data)}")
    
    corr_results = compute_episode_window_correlation(
        year_data, target_pv, cols_to_analyze
    )
    
    for col, (corr, pval, n) in corr_results.items():
        results_all.append({
            'Year': year,
            'Column': col,
            'Correlation': corr,
            'P_Value': pval,
            'N_Samples': n,
            'Significant': pval < 0.05,
        })
    
    print(f"    Computed correlations for {len(corr_results)} columns")

results_df = pd.DataFrame(results_all)
print(f"\n\nTotal correlation results: {len(results_df)}")

Total OP/PV rows: 1718039
Rows within episode windows: 78451
Percent within episodes: 4.57%
Years with OP/PV data inside episodes: [2022, 2023, 2024, 2025]

Processing Year: 2022
Rows in episode windows for 2022: 18758


    Computed correlations for 22 columns

Processing Year: 2023
Rows in episode windows for 2023: 11982


    Computed correlations for 22 columns

Processing Year: 2024
Rows in episode windows for 2024: 36976


    Computed correlations for 22 columns

Processing Year: 2025
Rows in episode windows for 2025: 10735


    Computed correlations for 22 columns


Total correlation results: 88


In [26]:
# Display top correlations (by absolute value) across all years
results_df['Abs_Correlation'] = results_df['Correlation'].abs()

top_corr = results_df.groupby('Column').agg({
    'Correlation': 'mean',
    'Abs_Correlation': 'mean',
    'P_Value': 'mean',
    'N_Samples': 'sum',
    'Significant': 'mean',
}).sort_values('Abs_Correlation', ascending=False)

print("Top 20 columns by average absolute correlation with 03LIC_1071.PV (episode windows):")
print("="*80)
top_corr.head(50)

Top 20 columns by average absolute correlation with 03LIC_1071.PV (episode windows):


,Correlation,Abs_Correlation,P_Value,N_Samples,Significant
Column,,,,,
03LIC_1016.PV,0.429852,0.429852,4.127385e-220,78431,1.00
03LIC_1071.OP,-0.417648,0.417648,6.793847e-35,78431,1.00
03LIC_1016.OP,-0.110051,0.187317,1.563998e-65,78431,1.00
03PIC_1013.OP,-0.169441,0.169441,3.693183e-69,78431,1.00
03FIC_3415.PV,0.141114,0.159080,1.221375e-12,77916,1.00
03FIC_3435.PV,-0.072947,0.140534,4.873416e-10,78317,1.00
03FIC_1085.OP,-0.007931,0.138143,2.340165e-07,78431,1.00
03LIC_1094.OP,0.005318,0.135729,2.605535e-14,78431,1.00
03FIC_3415.OP,0.075642,0.121413,6.443894e-21,78431,1.00


In [27]:
# Year-by-year comparison for top correlated columns
top_5_cols = top_corr.head(5).index.tolist()

print("Year-by-year correlation comparison for top 5 columns (episode windows):")
print("="*80)

for col in top_5_cols:
    print(f"\n{col}:")
    col_data = results_df[results_df['Column'] == col][['Year', 'Correlation', 'P_Value', 'N_Samples', 'Significant']]
    print(col_data.to_string(index=False))

Year-by-year correlation comparison for top 5 columns (episode windows):

03LIC_1016.PV:
 Year  Correlation       P_Value  N_Samples  Significant
 2022     0.469119  0.000000e+00      18758         True
 2023     0.283019 1.650954e-219      11982         True
 2024     0.603780  0.000000e+00      36958         True
 2025     0.363492  0.000000e+00      10733         True

03LIC_1071.OP:
 Year  Correlation      P_Value  N_Samples  Significant
 2022    -0.303433 0.000000e+00      18758         True
 2023    -0.111219 2.717539e-34      11982         True
 2024    -0.706684 0.000000e+00      36958         True
 2025    -0.549256 0.000000e+00      10733         True

03LIC_1016.OP:
 Year  Correlation       P_Value  N_Samples  Significant
 2022    -0.254201 1.968456e-274      18758         True
 2023     0.154532  6.255886e-65      11982         True
 2024    -0.091603  1.078110e-69      36958         True
 2025    -0.248931 2.677377e-151      10733         True

03PIC_1013.OP:
 Year  Correl

In [28]:
# Compare correlation consistency across years
print("Correlation consistency across years for top columns:")
print("="*80)

pivot_table = results_df[results_df['Column'].isin(top_5_cols)].pivot_table(
    index='Column',
    columns='Year',
    values='Correlation',
    aggfunc='mean',
)
pivot_table['Std_Across_Years'] = pivot_table.std(axis=1)
pivot_table

Correlation consistency across years for top columns:


Year,2022,2023,2024,2025,Std_Across_Years
Column,,,,,
03FIC_3415.PV,0.113953,0.248922,-0.035931,0.237511,0.132913
03LIC_1016.OP,-0.254201,0.154532,-0.091603,-0.248931,0.191843
03LIC_1016.PV,0.469119,0.283019,0.603780,0.363492,0.138752
03LIC_1071.OP,-0.303433,-0.111219,-0.706684,-0.549256,0.263189
03PIC_1013.OP,-0.127427,-0.183431,-0.198617,-0.168289,0.030624


In [29]:
# Specifically look at 03LIC_1071.OP correlation with 03LIC_1071.PV
print("03LIC_1071.OP correlation with 03LIC_1071.PV across years (episode windows):")
print("="*80)

op_corr = results_df[results_df['Column'] == '03LIC_1071.OP']
if len(op_corr) > 0:
    op_pivot = op_corr.pivot_table(
        index='Year',
        values='Correlation',
        aggfunc='mean',
    )
    print(op_pivot)
    print(f"\nMean correlation: {op_corr['Correlation'].mean():.4f}")
    print(f"All significant (p < 0.05): {op_corr['Significant'].all()}")
else:
    print("03LIC_1071.OP not found in results")

03LIC_1071.OP correlation with 03LIC_1071.PV across years (episode windows):
      Correlation
Year             
2022    -0.303433
2023    -0.111219
2024    -0.706684
2025    -0.549256

Mean correlation: -0.4176
All significant (p < 0.05): True


In [30]:
# Save results to CSV
output_path = 'RESULTS/episode_all_operator_action_plots/episode_window_correlation_analysis.csv'
results_df.to_csv(output_path, index=False)
print(f"Correlation results saved to: {output_path}")

# Summary statistics
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Total correlation pairs analyzed: {len(results_df)}")
print(f"Years analyzed: {valid_years}")
print(f"Significant correlations (p < 0.05): {results_df['Significant'].sum()} / {len(results_df)} ({results_df['Significant'].mean()*100:.1f}%)")

Correlation results saved to: RESULTS/episode_all_operator_action_plots/episode_window_correlation_analysis.csv

SUMMARY
Total correlation pairs analyzed: 88
Years analyzed: [2022, 2023, 2024, 2025]
Significant correlations (p < 0.05): 85 / 88 (96.6%)


In [31]:
# Print top 5 correlated tags for each year (episode windows)
print("Top 5 Correlated Tags with 03LIC_1071.PV by Year (episode windows)")
print("="*100)

for year in valid_years:
    print(f"\n{'='*50}")
    print(f"Year: {year}")
    print(f"{'='*50}")
    
    # Filter data for this year
    year_data = results_df[results_df['Year'] == year].copy()
    year_data['Abs_Correlation'] = year_data['Correlation'].abs()
    
    # Sort by absolute correlation
    year_data_sorted = year_data.sort_values('Abs_Correlation', ascending=False)
    
    # Get top 5
    top_5 = year_data_sorted.head(5)
    
    print(f"\n{'Rank':<6}{'Column':<20}{'Correlation':<15}{'P-Value':<15}{'N_Samples':<12}{'Significant'}")
    print("-"*80)
    
    for idx, (_, row) in enumerate(top_5.iterrows(), 1):
        sig_str = "Yes" if row['Significant'] else "No"
        print(f"{idx:<6}{row['Column']:<20}{row['Correlation']:>14.4f}{row['P_Value']:>14.2e}{row['N_Samples']:>12}{sig_str:>12}")
    
    # Summary statistics for this year
    print(f"\nYear {year} Summary:")
    print(f"  Total correlations computed: {len(year_data)}")
    print(f"  Significant correlations: {year_data['Significant'].sum()} ({year_data['Significant'].mean()*100:.1f}%)")
    print(f"  Mean absolute correlation: {year_data['Abs_Correlation'].mean():.4f}")
    print(f"  Max absolute correlation: {year_data['Abs_Correlation'].max():.4f}")

# Overall comparison across years
print(f"\n\n{'='*100}")
print("CROSS-YEAR COMPARISON: Top 5 Tags")
print(f"{'='*100}\n")

for col in top_5_cols:
    print(f"{col}:")
    col_by_year = results_df[results_df['Column'] == col][['Year', 'Correlation', 'P_Value', 'Significant']].sort_values('Year')
    print(col_by_year.to_string(index=False))
    print()

Top 5 Correlated Tags with 03LIC_1071.PV by Year (episode windows)

Year: 2022

Rank  Column              Correlation    P-Value        N_Samples   Significant
--------------------------------------------------------------------------------
1     03LIC_1016.PV               0.4691      0.00e+00       18758         Yes
2     03LIC_1071.OP              -0.3034      0.00e+00       18758         Yes
3     03LIC_1016.OP              -0.2542     1.97e-274       18758         Yes
4     03LIC_1085.OP              -0.1966     9.60e-163       18758         Yes
5     03FIC_1085.PV              -0.1952     6.33e-160       18691         Yes

Year 2022 Summary:
  Total correlations computed: 22
  Significant correlations: 21 (95.5%)
  Mean absolute correlation: 0.1380
  Max absolute correlation: 0.4691

Year: 2023

Rank  Column              Correlation    P-Value        N_Samples   Significant
--------------------------------------------------------------------------------
1     03LIC_1016.PV       